In [48]:
import pandas as pd
import lightgbm as lgb

In [21]:
all_train = pd.read_csv('train.csv')
testA = pd.read_csv('testA.csv')

In [25]:
all_data = pd.concat([all_train, testA])

In [30]:
all_data['isDefault'].value_counts(dropna=False)

0.0    640390
NaN    200000
1.0    159610
Name: isDefault, dtype: int64

In [33]:
all_train.tail()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
799995,799995,25000.0,3,14.49,860.41,C,C4,2659.0,7 years,1,...,6.0,2.0,12.0,13.0,10.0,14.0,0.0,0.0,0.0,3.0
799996,799996,17000.0,3,7.90,531.94,A,A4,29205.0,10+ years,0,...,15.0,16.0,2.0,19.0,2.0,7.0,0.0,0.0,0.0,0.0
799997,799997,6000.0,3,13.33,203.12,C,C3,2582.0,10+ years,1,...,4.0,26.0,4.0,10.0,4.0,5.0,0.0,0.0,1.0,4.0
799998,799998,19200.0,3,6.92,592.14,A,A4,151.0,10+ years,0,...,10.0,6.0,12.0,22.0,8.0,16.0,0.0,0.0,0.0,5.0
799999,799999,9000.0,3,11.06,294.91,B,B3,13.0,5 years,0,...,3.0,4.0,4.0,8.0,3.0,7.0,0.0,0.0,0.0,2.0


In [32]:
testA.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,800000,14000.0,3,10.99,458.28,B,B3,7027.0,10+ years,0,...,8.0,4.0,15.0,19.0,6.0,17.0,0.0,0.0,1.0,3.0
1,800001,20000.0,5,14.65,472.14,C,C5,60426.0,10+ years,0,...,1.0,3.0,3.0,9.0,3.0,5.0,0.0,0.0,2.0,2.0
2,800002,12000.0,3,19.99,445.91,D,D4,23547.0,2 years,1,...,1.0,36.0,5.0,6.0,4.0,12.0,0.0,0.0,0.0,7.0
3,800003,17500.0,5,14.31,410.02,C,C4,636.0,4 years,0,...,7.0,2.0,8.0,14.0,2.0,10.0,0.0,0.0,0.0,3.0
4,800004,35000.0,3,17.09,1249.42,D,D1,368446.0,< 1 year,1,...,11.0,3.0,16.0,18.0,11.0,19.0,0.0,0.0,0.0,1.0


# data info

In [5]:
all_train.head().T

,0,1,2,3,4
id,0,1,2,3,4
loanAmnt,35000.0,18000.0,12000.0,11000.0,3000.0
term,5,5,5,3,3
interestRate,19.52,18.49,16.99,7.26,12.99
installment,917.97,461.9,298.17,340.96,101.07
grade,E,D,D,A,C
subGrade,E2,D2,D3,A4,C2
employmentTitle,320.0,219843.0,31698.0,46854.0,54.0
employmentLength,2 years,5 years,8 years,10+ years,NaN
homeOwnership,2,0,0,1,1


In [8]:
all_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  800000 non-null  int64  
 1   loanAmnt            800000 non-null  float64
 2   term                800000 non-null  int64  
 3   interestRate        800000 non-null  float64
 4   installment         800000 non-null  float64
 5   grade               800000 non-null  object 
 6   subGrade            800000 non-null  object 
 7   employmentTitle     799999 non-null  float64
 8   employmentLength    753201 non-null  object 
 9   homeOwnership       800000 non-null  int64  
 10  annualIncome        800000 non-null  float64
 11  verificationStatus  800000 non-null  int64  
 12  issueDate           800000 non-null  object 
 13  isDefault           800000 non-null  int64  
 14  purpose             800000 non-null  int64  
 15  postCode            799999 non-nul

In [17]:
all_train.select_dtypes('int').columns

Index(['id', 'term', 'homeOwnership', 'verificationStatus', 'isDefault',
       'purpose', 'regionCode', 'initialListStatus', 'applicationType'],
      dtype='object')

In [19]:
all_train.select_dtypes('object').columns

Index(['grade', 'subGrade', 'employmentLength', 'issueDate',
       'earliesCreditLine'],
      dtype='object')

In [20]:
all_train.select_dtypes('float').columns

Index(['loanAmnt', 'interestRate', 'installment', 'employmentTitle',
       'annualIncome', 'postCode', 'dti', 'delinquency_2years', 'ficoRangeLow',
       'ficoRangeHigh', 'openAcc', 'pubRec', 'pubRecBankruptcies', 'revolBal',
       'revolUtil', 'totalAcc', 'title', 'policyCode', 'n0', 'n1', 'n2', 'n3',
       'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14'],
      dtype='object')

In [ ]:
date_cols = []
cat_cols = []
num_cols = []

# baseline model

In [60]:
baseline_train = all_train.select_dtypes(['int', 'float'])

In [67]:
X_train = baseline_train[:700000].drop('isDefault', axis = 1)
y_train = baseline_train[:700000]['isDefault']

In [68]:
X_test = baseline_train[700000:].drop('isDefault', axis = 1)
y_test = baseline_train[700000:]['isDefault']

In [69]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [86]:
params = {'device_type': 'gpu',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [87]:
clf = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                verbose_eval=50,
                callbacks=[lgb.early_stopping(stopping_rounds=10)])

Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.711989
[100]	valid_0's auc: 0.71822
[150]	valid_0's auc: 0.721612
[200]	valid_0's auc: 0.723417
[250]	valid_0's auc: 0.724592
[300]	valid_0's auc: 0.725209
Early stopping, best iteration is:
[290]	valid_0's auc: 0.725263
